# Changes over time

My target here is generate :  
1. the columns ('count', 'salary_min', 'min_experience_years', 'views','salary_max', 'num_vacancies'), grouped by date.
2. the same columns grouped by both city_name and date.

In [11]:
import pandas as pd
import numpy as np
import re
import json

In [4]:
# make a column for minimum_experience_years
def calc_min_experince_years(row):
    row.min_experience_years = int(re.findall('^[^0-9]*([0-9]+)[^0-9]*',row.min_experience_years)[0])
    return row

In [5]:
# first group them by date and sabe to file 
data = pd.read_csv('./data/generated/wazzuf/wuzzuf_data_formatedate.csv')

data.drop(['id','displayed_job_title','job_category_1','job_category_2','job_category_3','job_industry_1',
          'job_industry_2','job_industry_3','career_level','city_name','description'
          ,'job_requirements'], axis=1,inplace=True)

try :
    data.columns.values[ data.columns.values == 'experience_years'] = 'min_experience_years'
except:
    pass

data = data.apply(calc_min_experince_years, axis=1)

data['count'] = 1

grouped_data = data.groupby('post_date').agg({'salary_min':'median','salary_max':'median',
                                                            'num_vacancies':'sum','min_experience_years':'median',
                                                            'views':'sum','count':'count'})

data.to_csv('./data/generated/wazzuf/wuzzuf_data_grouped_by_date.csv')

In [53]:
# now group them by both date and city_name and save to files ( a file for each city).
data = pd.read_csv('./data/generated/wazzuf/wuzzuf_data_with_some_cities.csv')
data.drop(['id','displayed_job_title','job_category_1','job_category_2','job_category_3','job_industry_1',
          'job_industry_2','job_industry_3','career_level'], axis=1,inplace=True)

try :
    data.columns.values[ data.columns.values == 'experience_years'] = 'min_experience_years'
except:
    pass

data = data.apply(calc_min_experince_years, axis=1)

data['count'] = 1

grouped_data = data.groupby(['city_name','post_date']).agg({'salary_min':'median','salary_max':'median',
                                                            'num_vacancies':'sum','min_experience_years':'median',
                                                            'views':'sum','count':'count'})

for city_name in grouped_data.index.levels[0]:
    df = grouped_data.loc[city_name].apply(lambda x:(x-x.min())/(x.max()-x.min()), axis=0)
    df.to_csv('./data/generated/wazzuf/'+city_name+'.csv')

In [8]:
# and now normalize the grouped data to values between 0 and 1 , and save to file.
normal_data = grouped_data.apply(lambda x:(x-x.min())/(x.max()-x.min()), axis=0)
normal_data.to_csv('./data/generated/wazzuf/wuzzuf_data_normalized.csv')

-------------